Two of the functions have one or more places where you are to "FILL THIS IN"

# HW7
# Hannah Wang

## Problem 1

In [12]:
import numpy as np
import scipy.optimize
import statsmodels.api as sm

In [13]:
class GBM:

    def __init__(self,S0,r,sigma):
        self.S0 = 1
        self.r = 0.03
        self.sigma = 0.20

In [14]:
hw7dynamics = GBM(S0=1,r=0.03,sigma=0.20)

In [15]:
class Put:
    
    def __init__(self,K,T):
        self.K = K
        self.T = T

In [16]:
hw7contract = Put(K=1.1,T=4) 

In [17]:
class MC:
    
    def __init__(self,M,N,seed,algorithm):
        
        self.M = M # Number of paths  
        self.N = N     # Number of time periods  
        self.rng = np.random.default_rng(seed=seed) # Seeding the random number generator with a specified number helps make the calculations reproducible

        self.algorithm = algorithm   
        #'value' for Value-based approach (Longstaff-Schwartz) -- problem 1a
        #'policy' for Policy optimization -- problem 1b
    
    def price_americanPut_GBM(self,contract,dynamics):

        r=dynamics.r
        sigma=dynamics.sigma
        S0=dynamics.S0

        K=contract.K
        T=contract.T

        N=self.N
        M=self.M
        dt=T/N

        Z = self.rng.normal(size=(M,N))

        paths = S0*np.exp((r-sigma**2/2)*dt*np.tile(np.arange(1,N+1),(M,1))+sigma*np.sqrt(dt)*np.cumsum(Z,axis=1))

        payoffDiscounted = np.maximum(0,K-paths[:,-1])
        #This is the payoff (cashflow) along each path,
        #discounted to time nn (for nn=N,N-1,...)
        #It corresponds to the far right-hand column in each page of the
        #Excel worksheet
        #I'm initializing it for time nn=N.

        #You could make payoffDiscounted
        #to be a matrix because it depends on nn.
        #But I will just reuse a 1-dimensional array,
        #by overwriting the time nn+1 entries at time nn.        

        for nn in np.arange(N-1,0,-1):
            continuationPayoffDiscounted = np.exp(-r*dt)*payoffDiscounted
            # This is the CONTINUATION payoff (cashflow) along each path,
            # discounted to time nn (for nn=N-1,N-2,...)
            # It corresponds to the blue column in each page of the Excel worksheet
            # Note that payoffDiscounted comes from the previous iteration 
            # -- which was at time nn+1.  So now we discount back to time nn.

            X=paths[:,nn-1]               
            exerciseValue = K-X

            if self.algorithm == 'value': 
                # This is the value function (Longstaff-Schwartz) approach.  For problem 1a

                basisfunctions = np.stack([np.ones(M), X, X**2], axis=1)
                # FILL THIS IN.  You may use np.stack
                        # This will be an M-by-3 array containing the basis functions (Same ones as L7.9-7.10, and Excel)

                coefficients = sm.OLS(continuationPayoffDiscounted, basisfunctions).fit().params
                # FILL THIS IN  
                        # This will be an array of 3 estimated "betas".

                estimatedContinuationValue = basisfunctions  @ coefficients
                # FILL THIS IN with an array of length M. 
                        # This is similar to the Red column in Excel

                whichPathsToExercise = (exerciseValue >= np.maximum(estimatedContinuationValue,0))
                        #This is a length-M array of Booleans

            elif self.algorithm == 'policy':
                # This is the policy optimization approach to Reinforcement learning.  For problem 1b

                (a_opt,b_opt) = scipy.optimize.minimize(
                    negofMCaverageOfExpectedPayouts,(0,0),args=(X,exerciseValue,continuationPayoffDiscounted),method='Nelder-Mead').x
                    #Chose Nelder-Mead optimizer because it is generating reasonable results with minimal coding effort
                    #But gradient methods, done properly, usually run faster

                whichPathsToExercise = softExercise(X,a_opt,b_opt)>0.5
                    #FILL THIS IN, using the right-hand side of the last equation on the homework sheet 
                    #This obtains the hard exercise decision from the optimized soft exercise function
                    #It should be a length-M array of Booleans (as it was in the "value" approach.  
                    #But here it comes from the softExercise function)

            else:
                raise ValueError('Unknown algorithm type')


            payoffDiscounted[whichPathsToExercise] = exerciseValue[whichPathsToExercise]
            # FILL THIS IN -- see the "discounted cashflow along path" column in Excel 
            payoffDiscounted[np.logical_not(whichPathsToExercise)] = 0 
            # FILL THIS IN -- see the "discounted cashflow along path" column in Excel

        # The time-0 calculation needs no regression
        continuationPayoffDiscounted = np.exp(-r*dt)*payoffDiscounted;
        estimatedContinuationValue = np.mean(continuationPayoffDiscounted);
        putprice = max(K-S0,estimatedContinuationValue);

        return(putprice)
    

In [18]:
hw7MC_1 = MC(M=10000,N=4,seed=0,algorithm='value')

In [39]:
# for Policy optimization approach, problem 1b
#
# If b<<0 then this function essentially returns nearly 1 if X<a, or nearly 0 if X>a
# but with some smoothing of the discontinuity, using a sigmoid function, to help the optimizer

def softExercise(X,a,b):
    return 1/(1+np.exp(np.minimum(-b*(X-a), 709)))

In [40]:
# for Policy optimization approach, problem 1b

def negofMCaverageOfExpectedPayouts(coefficients, x, exercisePayoff, continuationPayoff):

    p = softExercise(x,*coefficients)    

    # p and exercisePayoff and continuationPayoff are all length-M arrays

    return -np.sum(p*exercisePayoff + (1-p)*continuationPayoff)
# FILL THIS IN

## You fill in, what to return.  It should be the negative of the expression inside the max() on the homework sheet.
## Need to take the negative because we are calling "minimize" but we want to do _maximization_

In [41]:
hw7MC_1.price_americanPut_GBM(hw7contract,hw7dynamics)

0.10039301505422576

In [42]:
hw7MC_2 = MC(M=10000,N=4,seed=0,algorithm='policy')
hw7MC_2.price_americanPut_GBM(hw7contract,hw7dynamics)

0.1000405946128757

## Problem 2 

### 2(a)

$$\begin{aligned}
\frac{\partial^2 F}{\partial u^2} = \mathbb{E} \big[exp(iuR_1+ivR_2)(i R_1)^2\big]\\
\frac{\partial^2 F}{\partial u^2}(0,0) = \mathbb{E} \big[(i R_1)^2\big] = \mathbb{E}\big[-R_1^2\big]\\
\end{aligned}$$

Similarly,
$$\begin{aligned}
\frac{\partial^2 F}{\partial v^2} = \mathbb{E} \big[exp(iuR_1+ivR_2)(i R_2)^2\big]\\
\frac{\partial^2 F}{\partial v^2}(0,0) = \mathbb{E} \big[(i R_2)^2\big] = \mathbb{E}\big[-R_2^2\big]\\
\end{aligned}$$

Thus,
$$\begin{aligned}
\mathbb{E} (R_1^2 + R_2^2) &= \mathbb{E} (R_1^2) + \mathbb{E}(R_2^2) \\
    &= -\frac{\partial^2 F}{\partial u^2}(0,0) -\frac{\partial^2 F}{\partial v^2}(0,0)
\end{aligned}$$

### 2(b)

$$\begin{aligned}
\phi(w) &= \mathbb{E} \big[exp(iw(4R_1-3R_2))\big] \\
    &= \mathbb{E} \big[exp(i4wR_1-i3wR_2)\big] \\
    &= F(4w,-3w)\\
\end{aligned}$$

### 2(c)

$$\begin{aligned}
G(x,y) &= \mathbb{E} \big[exp(ixR_3+iyR_4))\big] \\
    &= \mathbb{E} \big[exp(ixR_3)\big]\mathbb{E} \big[exp(iyR_4)\big]  \text{(Since R_3 and R_4 are independent)} \\ 
    &= F(x,0)F(0,y)\\
\end{aligned}$$